<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# Ejercicio de clasificación con vecinos cercanos (KNN)

Ejemplo de clasificación utilizando vecinos cercanos para la clasificación de clientes y determinar que servicio ofrecerle a cada uno<br>

v1.1

In [ ]:
import os
import platform

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import keras
from keras.models import Sequential
#from keras.utils import to_categorical  
from keras.utils.np_utils import to_categorical # Si esto no funciona, probar con el import anterior

# Recolectar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle">

In [ ]:
if os.access('drug200.csv', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/drug200.csv > drug200.csv
    else:
        !wget drug200.csv https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/drug200.csv

### `drug200.csv`:
El dataset **`drug200.csv`** contiene diferentes tipos de drogas que se le dan a pacientes relativo a su historial clínico. El objetivo es dado un nuevo paciente clasificarlo y determinar que droga es la más apropiada para el.<br> [Dataset source](https://www.kaggle.com/jeevanrh/drug200csv)

- **Age** --> edad, ejemplo 25
- **Sex** --> género, ejemplo F(femenino), M(masculino)
- **BP (Blood Pressure)** --> presión arterial, ejemplo HIGH(alta)
- **Cholesterol** --> colesterol, ejemplo normal (NORMAL)
- **Na / k** --> concentración de sodio/potasio en sangre, ejemplo 7.8
- **Drug** --> droga suministrada, ejemplo drugC

# Procesar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle">

In [ ]:
df = pd.read_csv("drug200.csv")
des = df.describe()
des.loc['Nan'] = df.isna().sum()
des.loc['%Nan'] = (df.isna().mean())*100
des

In [ ]:
df.head()

In [ ]:
print('Cantidad de datos en observacion:', df.shape[0])

# Explorar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle">

In [ ]:
# Exploramos que tan balanceado está el dataset,
# en cuantos casos se suministró cada droga
df['Drug'].value_counts()

In [ ]:
ax = sns.countplot(data=df, x="Drug")

Se puede observar que en la mayoría de los casos se suministra la drogaY oa la drogaX, es muy probable que el modelo siga esta tendencia

#### Transformar variables categóricas texto a clases numeradas

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

df_cod = df.copy()
df_cod['DrugY'] = df_cod['Drug'].apply(lambda x: 1 if x == 'drugY' else 0)
df_cod = df_cod.drop('Drug', axis=1)

In [ ]:
df_cod.head()

In [ ]:
ax = sns.countplot(data=df_cod, x="DrugY")

In [ ]:
def one_hot_encoding(df, column):
    df_copy = df.copy()
    # LabelEncoder
    le = LabelEncoder()
    label_encoding = le.fit_transform(df_copy[column])
    # OneHotEncoder
    onehot_encoder = OneHotEncoder(sparse=False)
    one_hot_encoding = onehot_encoder.fit_transform(label_encoding.reshape(-1, 1))
    # Crear las columnas con el resultado del encoder
    one_hot_encoding_df = pd.DataFrame(one_hot_encoding, columns=le.classes_, dtype=int)
    # Agregar sufijo
    one_hot_encoding_df = one_hot_encoding_df.add_prefix(column+'_')
    # Unir nuevas columnas al dataset
    df_copy = df_copy.join(one_hot_encoding_df)
    # Eleminar vieja columna del dataset
    df_copy = df_copy.drop([column], axis=1)
    return df_copy


In [ ]:
df_cod = one_hot_encoding(df_cod, 'Sex')
df_cod = one_hot_encoding(df_cod, 'BP')
df_cod = one_hot_encoding(df_cod, 'Cholesterol')

In [ ]:
df_cod.head()

In [ ]:
sns.displot(data=df_cod, x='Age')
sns.displot(data=df_cod, x='Na_to_K')
plt.show()

In [ ]:
# Es importante normalizar los datos, ya que las redes funcionan con el proceso
# del gradiente descendente.
# Como "Age" no sigue una distribucion normal y "Na_to_K" no parece tener outliers
# utilizaremos el MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_norm = df_cod.copy()
df_norm.loc[:, 'Age'] = scaler.fit_transform(df_norm[['Age']])
df_norm.loc[:, 'Na_to_K'] = scaler.fit_transform(df_norm[['Na_to_K']])
df_norm.head()

# Entrenar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle">

El primer paso es obtener los datos que serán la entrada del sistema (X) y los datos que serán la salida del modelo estimador (y)

In [ ]:
X = df_norm.drop('DrugY', axis=1).values
y = df_norm['DrugY'].values
in_shape = X.shape[1]
out_shape = 1

Siguiente paso es dividir el dataset en entrenamiento (train) y evaluación (test). Utilizaremos el criterio 80%20%

In [ ]:
from sklearn.model_selection import train_test_split
# Fijamos un "random_state" constante para que siempre el dataset se parta de la misma forma
# para poder repetir los ensayos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from keras.layers import Dense

def create_model(in_shape, hidden_neurons, out_shape):
    # Crear un modelo secuencial
    model = Sequential()

    # Crear la capa de entrada y la capa oculta (hidden) de la red, que tendrá:
    # --> tantas entradas (in_shape) como columnas
    # --> tantas neuronas como deseemos
    # --> utilizamos "sigmoid" como capa de activación
    model.add(Dense(units=hidden_neurons, activation='sigmoid', input_shape=(in_shape,)))

    # Crear la capa de salida, que tendrá tantas neuronas como salidas posibles
    model.add(Dense(units=out_shape, activation='sigmoid'))

    return model

In [ ]:
model = create_model(in_shape, 64, out_shape)

model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, validation_split=0.2 , epochs=100, batch_size=32)

In [ ]:
epoch_count = range(1, len(history.history['accuracy']) + 1)
sns.lineplot(x=epoch_count,  y=history.history['accuracy'], label='train')
sns.lineplot(x=epoch_count,  y=history.history['val_accuracy'], label='valid')
plt.show()

In [ ]:
y_hat_prob = model.predict(X_test)
y_hat = [1 if x >= 0.5 else 0 for x in y_hat_prob]

In [ ]:
model.summary()

# Validar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline5.png" width="1000" align="middle">

In [ ]:
# Calcular la exactitud (accuracy)
scores = model.evaluate(X_test, y_test)
scores[1]

In [ ]:
# Calcular la exactitud (accuracy)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_hat, normalize=True)


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test, y_hat)
cmd = ConfusionMatrixDisplay(cm, display_labels=['NOT_Y', 'Y'])
cmd.plot(cmap=plt.cm.Blues)
plt.show()